SENT TO ME BY YAEL B.H.

In [1]:
from qiskit import IBMQ
import os
import csv
import datetime
IBMQ.load_account()
provider = IBMQ.get_provider(hub= 'ibm-q')
log_time = datetime.datetime.now()
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

backends = list(map( lambda x: x.name(), provider.backends()))[1:]


https://qiskit.org/documentation/stubs/qiskit.providers.ibmq.IBMQBackend.properties.html?highlight=backend%20properties#qiskit.providers.ibmq.IBMQBackend.properties

https://qiskit.org/documentation/stubs/qiskit.providers.models.BackendProperties.html

In [2]:
#avoid qasm_simulator


def write_log_to_file(csv_writer):
    for backend in map(lambda x: provider.get_backend(x),backends):
        try:
            """
            Class representing name-date-unit-value
            Attributes:
                date: date (datetime.datetime)
                name: name. (str)
                unit: unit. (str)
                value: value. (float)
            qubit_idx is a single digit integer; 
            qubit_prop is a list of Nduv
            """
            for qubit_idx, qubit_prop in enumerate(backend.properties(refresh=True).qubits):
                for prop in qubit_prop:
                    csv_writer.writerow([backend.name(), qubit_idx, prop.date.isoformat(), prop.name, prop.unit, prop.value, log_time.isoformat()])
                id_gate = [i for i in backend.properties().gates if i.gate =='id' and qubit_idx in i.qubits][0]
                id_gate_length = [p for p in id_gate.parameters if p.name=='gate_length'][0]
                csv_writer.writerow([backend.name(), qubit_idx, id_gate_length.date.isoformat(), 'id_length',
                                     id_gate_length.unit, id_gate_length.value, log_time.isoformat()])
        except Exception as e:
            print("Cannot add backend", backend.name(), e)


In [3]:

absolute_path = 'C:\\Users\\Annanay\\OneDrive\\Documents\\Work\\project\\data_and_scripts\\' + log_time.strftime("%d") + '_' + log_time.strftime('%m') + '.csv'
if not os.path.isfile(absolute_path):
    with open(absolute_path, "w") as f:
        csv_writer = csv.writer(f)
        csv_writer.writerow(["backend","qubit","datetime","name","units","value","log_datetime"])
        write_log_to_file(csv_writer)
else :
    with open(absolute_path, "a") as f:
        write_log_to_file(csv.writer(f))


In [ ]:
"""
Get update schedules for each backend for last 1 month. Heirarchy:
Backend --- qubits --- all qubit properties
                  --- gates --- gate_error, gate_length
                  
Could definitely be made more efficient by ensuring no duplicates are added
(rather than just dropping them at the end)
"""
current_time = datetime.datetime.now()
prev_time = datetime.datetime(2020,5,20,12,0,0)
results = pd.DataFrame({}, columns=['backend','qubit', 'gate','prop_name', 'prop_time'])

while current_time > prev_time: 
    for backend in map(lambda x: provider.get_backend(x),backends):
        try:
                """
                Class representing name-date-unit-value
                Attributes:
                    date: date (datetime.datetime)
                    name: name. (str)
                    unit: unit. (str)
                    value: value. (float)
                qubit_idx is a single digit integer; 
                qubit_prop is a list of Nduv
                """
                PROPS = backend.properties(refresh=True, datetime = current_time)
                for qubit_idx, qubit_prop in enumerate(PROPS.qubits):
                    #qubit properties first
                    results = results.append(pd.DataFrame({
                        'backend': [backend.name() for i in qubit_prop],
                        'qubit': [qubit_idx for i in qubit_prop],
                        'gate': [None for i in qubit_prop],
                        'prop_name': [prop.name for prop in qubit_prop],
                        'prop_time': [prop.date for prop in qubit_prop]
                    }, columns=['backend','qubit', 'gate','prop_name', 'prop_time']))

                    #then gate properties. Note that 2-qubit gates get a repeat entry
                    #(one for each qubit they act on)
                    for gate in [i for i in PROPS.gates if qubit_idx in i.qubits]:
                        results = results.append(pd.DataFrame({
                            'backend': [backend.name() for i in gate.parameters],
                            'qubit': [qubit_idx for i in gate.parameters],
                            'gate': [gate.gate for i in gate.parameters],
                            'prop_name': [i.name for i in gate.parameters],
                            'prop_time': [i.date for i in gate.parameters]
                        }, columns=['backend','qubit', 'gate','prop_name', 'prop_time']))

        except Exception as e:
            print("Cannot add backend", backend.name(), current_time, e)


        current_time -= datetime.timedelta(seconds = 1800)

results.drop_duplicates(inplace=True)
results.to_csv('update_schedule.csv',index=False)

In [ ]:
def convert_to_datetime(bad_str):
    """
    Converts the muddled time data strings from CSV to datetime objects
    """
    try:
        try:
            return (datetime.datetime.strptime(str(bad_str)[:str(bad_str).index('+')], '%Y-%m-%dT%H:%M:%S'))
        except:
            return (datetime.datetime.strptime(str(bad_str)[:str(bad_str).index('+')], '%Y-%m-%d %H:%M:%S'))
    except Exception as e:
        print('ERROR ERROR', bad_str, e)

results = pd.read_csv('update_schedule.csv')
results['prop_time']= results['prop_time'].apply(convert_to_datetime)

In [ ]:
    """
    CHANGE PROPERTIES HERE
    """
plot = results[(results['backend'] == 'ibmqx2') & (results['prop_name'] == 'gate_length') & (results['qubit'] == 4)].reset_index(drop=True).iloc[::-1]
plot['timestamp'] = plot['prop_time'].apply(lambda x: datetime.datetime.timestamp(x))
del plot['prop_name']
del plot ['prop_time']
plot.to_csv('plot1.csv', index=False)

In [ ]:
from operator import itemgetter
from collections import defaultdict


def plot_timeline(dataset, **kwargs):
    """
    Plots a timeline of events from different sources to visualize a relative
    sequence or density of events. Expects data in the form of:
        (timestamp, source, category {colour})
    Though this can be easily modified if needed. Expects sorted input.
    Taken from https://bbengfort.github.io/snippets/2016/01/29/timeline-visualization.html
    """
    outpath = kwargs.pop('savefig', None)  # Save the figure as an SVG
    colors  = kwargs.pop('colors', {})     # Plot the colors for the series.
    series  = set([])                      # Figure out the unique series

    # Bring the data into memory and sort
    dataset = list(dataset)

    # Make a first pass over the data to determine number of series, etc.
    for _, source, category in dataset:
        series.add(source)
        if category not in colors:
            colors[category] = 'k'

    # Sort and index the series
    series  = sorted(list(series))

    # Create the visualization
    x = []  # Scatterplot X values
    y = []  # Scatterplot Y Values
    c = []  # Scatterplot color values

    # Loop over the data a second time
    for timestamp, source, category in dataset:
        x.append(datetime.datetime.fromtimestamp(timestamp))
        y.append(series.index(source))
        c.append(colors[category])

    plt.figure(figsize=(14,4))
    """
    CHANGE TITLE HERE
    """
    plt.title(kwargs.get('title', "Qubit 4 Gate Length Update Schedule"))
    #plt.ylim((-1,len(series)))
    #plt.xlim((-1000, dataset[-1][0]+1000))
    plt.yticks(range(len(series)), series)
    plt.scatter(x, y, color=c, alpha=0.85, s=10)

    if outpath:
        return plt.savefig(outpath, format='svg', dpi=1200)

    return plt


colors = {'ibmqx2': 'r'}
with open('plot1.csv', 'r') as f:
    reader = csv.reader(f)
    plt = plot_timeline([
        (float(row[0]), row[1], row[2])
        for row in reader
    ], colors=colors)
plt.gca().xaxis.grid(True)
plt.show()
